## Example 2 - Plot PSDs for a station
The intent of this series of Jupyter Notebooks is to demonstrate how metrics can be retrieved from an ISPAQ sqlite database and provide some ideas on how to use or plot those metrics.  

This example plots station PSDs. It requires that we have the PSD values already calculated for the target for the requested
days, and those values should live in the example database, named ispaq_example.db.  If you have not already, you can run this command in your ISPAQ conda environment
to have the values generated for the target-days in this example (it will take several minutes to run):

    python3 run_ispaq.py -M psd_corrected -S ANMO --starttime 2020-10-01 --endtime 2020-10-16 --output db --db_name ispaq_example.db

This example will assume that the above command has already been run and the PSDs already exist.

To begin, we need to import the necessary modules:

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import datetime

Now we need to set some variables.

In [ ]:
filename = 'PSD.png'
db_name = '../ispaq_example.db'
metric = 'psd_corrected'
startDate = '2020-10-01'
endDate = '2020-10-15'
target = 'IU.ANMO.00.BH1.M'
filename = f'example2_{target}_{startDate}_{endDate}_PSD.png'
filename2 = f'example2_{target}_{startDate}_{endDate}_PSD_bw.png'

The first step is to create a query that will be used to retrieve the psds.

In [ ]:
SQLcommand = "SELECT * FROM " + metric + \
                     " WHERE start >= '" + startDate + "' " \
                     "and start < '" + endDate + "' " \
                     "and (target like '" + target + "');"

print('\nThis is the query used to retrieve the PSDs from the database:')
print(SQLcommand)

Create a connection to the database and run the query, loading it into a pandas dataframe

In [ ]:
try:
    conn = sqlite3.connect(db_name)
    DF = pd.read_sql_query(SQLcommand, conn, parse_dates=['start','end'])
    conn.close
except:
    print(f"Unable to connect to or find the {metric} table in the database {db_name}")

At this point, we have created a query to retrieve the metrics from the SQLite database, used sqlite3 to connect to the database, retreieved the metrics, closed the connection, and then ensured that the start times are in a datetime format for plotting purposes.

This is what the dataframe looks like:

In [ ]:
print(DF)

The PSD plot will be power vs. frequency, so we are going to group them together by start time:

In [ ]:
DF = DF[['frequency','power','start']]
DFgrouped = DF.groupby(['start'])
print(DFgrouped)

We will take the information from the dataframe that we loaded and rearrange it for plotting.

In [ ]:
plotDF = pd.DataFrame()
for name, group in DFgrouped:
    tmpDF = pd.DataFrame()
    tmpDF[name] = group['power']
    tmpDF.set_axis(group['frequency'], axis='index', inplace=True)

    plotDF = pd.concat([plotDF, tmpDF], axis=1, sort=False)

print("\nThis is the dataframe that will be used to plot:")
print(plotDF)

Now that we have the dataframe in this arrangement, we can start plotting it up.

First, a colorful version of the plot:

In [ ]:
ax = plotDF.plot(legend=False, alpha=1, title=f'{target}\n{startDate} through {endDate}')
ax.set_xscale('log')
ax.invert_xaxis()
ax.set_ylabel('power')
plt.grid(True)
plt.savefig(filename)

Then, just for the sake of variety, a black and white version of the version of the plot:

In [ ]:
ax2 = plotDF.plot(legend=False, alpha=.02, color='k', title=f'{target}\n{startDate} through {endDate}')
ax2.set_xscale('log')
ax2.invert_xaxis()
ax2.set_ylabel('power')
plt.grid(True)
plt.savefig(filename2)